In [3]:
import pandas as pd
import numpy as np
import random
import math
import time
import collections
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn import tree as sk_tree

In [31]:
if __name__ == '__main__':
    dataFrame = pd.read_csv(f'genbase_cvt.txt',sep=",")
    dataFrame = dataFrame.drop('protein', axis=1)
    # dataFrame = pd.DataFrame({
    #     "Outlook":
    #     ["S","S","O","R","R","R","O","S","S","R","S","O","O","R"],
    #     "Temp": 
    #     ["H","H","H","M","C","C","C","M","C","M","M","M","H","M"],
    #     "Humidity": 
    #     ["H","H","H","H","N","N","N","H","N","N","N","H","N","H"],
    #     "Windy":
    #     ["F","T","F","F","F","T","T","F","F","F","T","T","F","T"],
    #     "Play":
    #     ["F","F","T","T","T","F","T","F","T","T","T","T","T","F"]
    # })

    # create LabelEncoder object
    label_encoder = LabelEncoder()

    # fit and transform the categorical variable

    # for header in column_headers:
    for header in dataFrame.columns:
      dataFrame[header] = label_encoder.fit_transform(dataFrame[header])

    x, y = dataFrame.iloc[:, :-27], dataFrame.iloc[:, -27:]
    x_train, x_test, y_train, y_test = train_test_split(
      x, y, test_size=0.3, random_state=random.randint(1, 25)
    )

    print(x_train,"\n\n\n",y_train)

      PS00010   PS00011   PS00012   PS00014   PS00017   PS00018   PS00019  \
158         0         0         0         0         0         0         0   
344         0         0         0         0         0         0         0   
415         0         0         0         0         0         0         0   
28          0         0         0         0         0         0         0   
139         0         0         0         0         0         0         0   
..        ...       ...       ...       ...       ...       ...       ...   
337         0         0         0         0         0         0         0   
91          0         0         0         0         0         0         0   
80          0         0         0         0         0         0         0   
191         0         0         0         0         0         0         0   
409         0         0         0         0         1         0         0   

      PS00020   PS00021   PS00022  ...   PS50821   PS50822   PS50823  \
158

# Code below for multiple label dataset (can be multiple value in single label) with joint information gain

Marginal entropy

In [32]:
def get_marginal_ent(y):
  mgn_entropys = []
  for class_header in y.columns:
    y_label = y[class_header].value_counts()
    mgn_entropy = 0
    for idx in range(len(y_label)):
      prob = y_label[idx]/len(y)
      mgn_entropy += (prob * math.log2(prob))
    mgn_entropys.append(-1*mgn_entropy)
    print(f"marginal entropy {class_header}: {mgn_entropy}\n")
  return mgn_entropys

Conditional entropy

In [33]:
def get_conditional_entropy(x,y):

  cond_entropys = []
  attrs = x.columns
  class_label_name = y.name

  for attr in attrs:
    
    # count all unique value of an attribute
    attr_vals = x[attr].value_counts()
    cond_entropy = 0

    for attr_val in range(len(attr_vals)):

      # print attribute name and the selected value 
      # print(attr, attr_val)

      # get index of specific attribute value e.g idx of (Outlook == Sunny)
      val_idx = x[x[attr]==attr_val].index
      # print(val_idx)

      # total count of instance with specific attribute value
      val_cnt = len(val_idx)
      val_prob = -1*val_cnt/len(y)

      # get information of class label based on specific attr value idx
      val_labels = y[val_idx].value_counts()
      # print(val_labels)
      tmp = 0

      for val_label_idx, val_label in enumerate(val_labels):
        if (val_label == val_cnt):
          tmp += 0
        else :
          val_label_prob = val_label/val_cnt 
          tmp += (val_label_prob* math.log2(val_label_prob))

        # count total label for each attr value
        # print(f"label {val_label_idx} count" \
        #   f" in attr {attr}-{attr_val} : {val_label}"
        # )
        
      cond_entropy += val_prob * tmp
      # print(f"{val_prob * tmp}\n")

    print(f"conditional entropy for {class_label_name}|{attr}:",cond_entropy)
    print("\n")
    cond_entropys.append(cond_entropy)

  print(cond_entropys)
  return cond_entropys

Get Joint Information Gain

In [34]:
# Get marginal entropy for each class
mgn_entropys = get_marginal_ent(y)

joint_cond_entropys = []
# Get conditional entropy for each class label
for class_header_idx,class_header in enumerate(y.columns):
  #y_label = y[class_header].value_counts()
  y_label = y[class_header]
  joint_cond_entropys.append(get_conditional_entropy(x,y_label))

# Get Joint Information gain for all attribute/feature
joint_information_gains = []
for cond_idx, cond_entropys in enumerate(joint_cond_entropys[0]):
  joint_information_gain = 0
  for mgn_idx,mgn_entropy in enumerate(mgn_entropys):
    joint_information_gain += mgn_entropy-joint_cond_entropys[mgn_idx][cond_idx]
  joint_information_gains.append(joint_information_gain)
np_joint_information_gains = np.array(joint_information_gains)

Streaming output truncated to the last 5000 lines.
conditional entropy for  PDOC00653| PS01143: 0.0


conditional entropy for  PDOC00653| PS01148: 0.0


conditional entropy for  PDOC00653| PS01149: 0.0


conditional entropy for  PDOC00653| PS01152: 0.0


conditional entropy for  PDOC00653| PS01154: 0.0


conditional entropy for  PDOC00653| PS01156: 0.0


conditional entropy for  PDOC00653| PS01157: 0.0


conditional entropy for  PDOC00653| PS01159: 0.0


conditional entropy for  PDOC00653| PS01162: 0.0


conditional entropy for  PDOC00653| PS01163: 0.0


conditional entropy for  PDOC00653| PS01166: 0.0


conditional entropy for  PDOC00653| PS01167: 0.0


conditional entropy for  PDOC00653| PS01168: 0.0


conditional entropy for  PDOC00653| PS01169: 0.0


conditional entropy for  PDOC00653| PS01173: 0.0


conditional entropy for  PDOC00653| PS01174: 0.0


conditional entropy for  PDOC00653| PS01176: 0.0


conditional entropy for  PDOC00653| PS01177: 0.0


conditional entropy for  PDOC00

In [39]:
print("all class marginal entropy:\n",len(mgn_entropys),mgn_entropys)
print("all class conditional entropy:\n",np.array(joint_cond_entropys).shape)
print(joint_cond_entropys)
print("joint ig for all attr:\n",np.array(joint_information_gains))
print("max joint ig on attribute\n",
      x.columns[np_joint_information_gains.argmax()], "\nwith joint ig value:",
      np_joint_information_gains[np_joint_information_gains.argmax()]
      )

all class marginal entropy:
 27 [0.5377028326541131, 0.5510769294648428, 0.4022904876236095, 0.393840774371116, 0.907726176870597, 0.25067362013882866, 0.3217619381326815, 0.48109322142604566, 0.08287072413939012, 0.26138961733912397, 0.21717621039864946, 0.15570768679078706, 0.393840774371116, 0.2055074352347435, 0.19355410182862173, 0.06576492192087594, 0.19355410182862173, 0.33124873828887086, 0.15570768679078706, 0.11396732231829328, 0.06576492192087594, 0.046916068950638004, 0.06576492192087594, 0.046916068950638004, 0.023461855377690657, 0.023461855377690657, 0.023461855377690657]
all class conditional entropy:
 (27, 1185)
[[0.5377028326541131, 0.5377028326541131, 0.5377028326541131, 0.5346897626155589, 0.5315654711067958, 0.534361326754992, 0.5377028326541131, 0.5377028326541131, 0.5377028326541131, 0.5377028326541131, 0.5377028326541131, 0.5377028326541131, 0.5377028326541131, 0.5377028326541131, 0.5372887414135844, 0.5377028326541131, 0.5377028326541131, 0.5377028326541131, 0.

# Code below for single label dataset (can be multiple value in single label) with information gain

In [ ]:
print(y.value_counts())
print(len(y))
mgn_entropy = 0
cond_entropys = []
for idx in range(len(y.value_counts())):
  prob = y.value_counts()[idx]/len(y)
  print(prob * math.log2(prob))
  mgn_entropy += (prob * math.log2(prob))

print("marginal entropy: "+str(-1*mgn_entropy),"\n")

# print(x.Outlook.value_counts())
# print(x[x.columns[0]])

# count number of column in feature spaces
attrs = x.columns
for attr in range(len(attrs)):

  # count all unique value of an attribute
  attr_vals = x[x.columns[attr]].value_counts()
  cond_entropy = 0

  for attr_val in range(len(attr_vals)):
    
    # print attribute name and the selected value 
    print(x.columns[attr], attr_val)

    # get index of specific attribute value e.g idx of (Outlook == Sunny)
    val_idx = x[x[x.columns[attr]]==attr_val].index
    print(val_idx)

    # total count of instance with specific attribute value
    val_cnt = len(val_idx)
    val_prob = -1*val_cnt/len(y)

    # get information of class label based on specific attr value idx
    val_labels = y[val_idx].value_counts()
    print(val_labels)
    tmp = 0

    for val_label_idx, val_label in enumerate(val_labels):
      if (val_label == val_cnt):
        tmp += 0
      else :
        val_label_prob = val_label/val_cnt 
        tmp += (val_label_prob* math.log2(val_label_prob))

      # count total label for each attr value
      print(f"label {val_label_idx} count" \
        f" in attr {x.columns[attr]}-{attr_val} : {val_label}"
      )
    cond_entropy += val_prob * tmp
    print(f"{val_prob * tmp}\n")

  print(f"conditional entropy for Class|{x.columns[attr]}:",cond_entropy)
  print("\n")
  cond_entropys.append(cond_entropy)

print(cond_entropys)

for cond_entropy in cond_entropys:
  print(-1*mgn_entropy-cond_entropy)

 PDOC00154   PDOC00343   PDOC00271   PDOC00064   PDOC00791   PDOC00380   PDOC50007   PDOC00224   PDOC00100   PDOC00670   PDOC50002   PDOC50106   PDOC00561   PDOC50017   PDOC50003   PDOC50006   PDOC50156   PDOC00662   PDOC00018   PDOC50001   PDOC00014   PDOC00750   PDOC50196   PDOC50199   PDOC00660   PDOC00653   PDOC00030
0           0           0           0           1           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0             170
1           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0              73
0           1    

TypeError: ignored

# RF Class

In [ ]:
# -*- coding: utf-8 -*-
"""
@Env: Python2.7
@Time: 2019/10/24 13:31
@Author: zhaoxingfeng
@Function：Random Forest（RF），随机森林二分类
@Version: V1.2
参考文献：
[1] UCI. wine[DB/OL].https://archive.ics.uci.edu/ml/machine-learning-databases/wine.
"""

class Tree(object):
    """Define a decision tree"""
    def __init__(self):
        self.split_feature = None
        self.split_value = None
        self.leaf_value = None
        self.tree_left = None
        self.tree_right = None

    def calc_predict_value(self, dataset):
        """Find the leaf node of the sample through the recursive decision tree"""
        if self.leaf_value is not None:
            return self.leaf_value
        elif dataset[self.split_feature] <= self.split_value:
            return self.tree_left.calc_predict_value(dataset)
        else:
            return self.tree_right.calc_predict_value(dataset)

    def describe_tree(self):
        """
        Print the decision tree in json form, 
        which is convenient for viewing the tree structure
        """
        if not self.tree_left and not self.tree_right:
            leaf_info = "{leaf_value:" + str(self.leaf_value) + "}"
            return leaf_info
        left_info = self.tree_left.describe_tree()
        right_info = self.tree_right.describe_tree()
        tree_structure = "{split_feature:" + str(self.split_feature) + \
                         ",split_value:" + str(self.split_value) + \
                         ",left_tree:" + left_info + \
                         ",right_tree:" + right_info + "}"
        return tree_structure

In [ ]:
class RandomForestClassifier(object):
    def __init__(self, n_estimators=10, max_depth=-1, min_samples_split=2, min_samples_leaf=1,
                 min_split_gain=0.0, colsample_bytree=None, subsample=0.8, random_state=None):
        """
        Random Forest Parameters
         ----------
         n_estimators: 
              number of trees
         max_depth: 
              tree depth, -1 means unlimited depth
         min_samples_split: 
              The minimum number of samples required for node splitting, 
              the node terminates splitting if it is less than this value
         min_samples_leaf: 
              The minimum sample number of leaf nodes, 
              less than this value leaves are merged
         min_split_gain: 
              The minimum gain required for splitting, 
              less than this value the node terminates the split
         colsample_bytree: 
              Column sampling setting, which can be [sqrt, log2]. 
              sqrt means randomly selecting sqrt(n_features) features,
              log2 means to randomly select log(n_features) features, 
              if set to other, column sampling will not be performed
         subsample: 
              line sampling ratio
         random_state: 
              Random seed, after setting, 
              the n_estimators sample sets generated each time will not change, 
              ensuring that the experiment can be repeated
        """
        self.n_estimators = n_estimators
        self.max_depth = max_depth if max_depth != -1 else float('inf')
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.min_split_gain = min_split_gain
        self.colsample_bytree = colsample_bytree
        self.subsample = subsample
        self.random_state = random_state
        self.trees = None
        self.feature_importances_ = dict()

    def fit(self, dataset, targets):
        """Model training entry"""
        assert targets.unique().__len__() >= 2, "There must be two class for targets!"
        targets = targets.to_frame(name='label')

        if self.random_state:
            random.seed(self.random_state)
        random_state_stages = random.sample(range(self.n_estimators), self.n_estimators)

        # Two column sampling methods
        if self.colsample_bytree == "sqrt":
            self.colsample_bytree = int(len(dataset.columns) ** 0.5)
        elif self.colsample_bytree == "log2":
            self.colsample_bytree = int(math.log(len(dataset.columns)))
        else:
            self.colsample_bytree = len(dataset.columns)

        # Build multiple decision trees in parallel
        self.trees = Parallel(n_jobs=-1, verbose=0, backend="threading")(
            delayed(self._parallel_build_trees)(dataset, targets, random_state)
                for random_state in random_state_stages)
        
    def _parallel_build_trees(self, dataset, targets, random_state):
        """
        bootstrap has put back sampling to 
        generate a training sample set and build a decision tree
        """
        subcol_index = random.sample(dataset.columns.tolist(), self.colsample_bytree)
        dataset_stage = dataset.sample(n=int(self.subsample * len(dataset)), replace=True, 
                                        random_state=random_state).reset_index(drop=True)
        dataset_stage = dataset_stage.loc[:, subcol_index]
        targets_stage = targets.sample(n=int(self.subsample * len(dataset)), replace=True, 
                                        random_state=random_state).reset_index(drop=True)

        tree = self._build_single_tree(dataset_stage, targets_stage, depth=0)

        # -------------- PRINT BEST NODE --------------
        # print(tree.describe_tree())

        return tree

    def _build_single_tree(self, dataset, targets, depth):
        """Recursively build a decision tree"""
        # If the categories of the node 
        # are all the same/the samples are less than 
        # the minimum number of samples required for splitting, 
        # select the category with the most occurrences. 
        # Termination of division/split
        if len(targets['label'].unique()) <= 1 or dataset.__len__() <= self.min_samples_split:
            tree = Tree()
            tree.leaf_value = self.calc_leaf_value(targets['label'])
            return tree

        if depth < self.max_depth:
            best_split_feature, best_split_value, best_split_gain = self.choose_best_feature(dataset, targets)
            left_dataset, right_dataset, left_targets, right_targets = \
                self.split_dataset(dataset, targets, best_split_feature, best_split_value)

            tree = Tree()
            # If after the parent node is split, 
            # the left leaf node/right leaf node sample is less than 
            # the set minimum number of leaf node samples, 
            # the parent node will terminate the split
            if left_dataset.__len__() <= self.min_samples_leaf or \
                    right_dataset.__len__() <= self.min_samples_leaf or \
                    best_split_gain <= self.min_split_gain:
                tree.leaf_value = self.calc_leaf_value(targets['label'])
                return tree
            else:
                # If this feature is used when splitting, 
                # the importance of this feature will be increased by 1
                self.feature_importances_[best_split_feature] = \
                    self.feature_importances_.get(best_split_feature, 0) + 1

                tree.split_feature = best_split_feature
                tree.split_value = best_split_value
                tree.tree_left = self._build_single_tree(left_dataset, left_targets, depth+1)
                tree.tree_right = self._build_single_tree(right_dataset, right_targets, depth+1)
                return tree
        # If the depth of the tree exceeds the preset value, terminate the split
        else:
            tree = Tree()
            tree.leaf_value = self.calc_leaf_value(targets['label'])
            return tree

    def choose_best_feature(self, dataset, targets):
        """
        Find the best data set division method, 
        find the optimal split feature, 
        split threshold, split gain
        """
        best_split_gain = 1
        best_split_feature = None
        best_split_value = None

        for feature in dataset.columns:
            if dataset[feature].unique().__len__() <= 100:
                unique_values = sorted(dataset[feature].unique().tolist())
            # If the dimension feature has too many values, 
            # select the 100th percentile value as the split threshold to be selected
            else:
                unique_values = np.unique([np.percentile(dataset[feature], x)
                                           for x in np.linspace(0, 100, 100)])

            # Calculate the splitting gain for the possible splitting thresholds, 
            # and select the threshold with the largest gain
            for split_value in unique_values:
                left_targets = targets[dataset[feature] <= split_value]
                right_targets = targets[dataset[feature] > split_value]
                split_gain = self.calc_gini(left_targets['label'], right_targets['label'])

                if split_gain < best_split_gain:
                    best_split_feature = feature
                    best_split_value = split_value
                    best_split_gain = split_gain
        return best_split_feature, best_split_value, best_split_gain

    @staticmethod
    def calc_leaf_value(targets):
        """
        Select the category with the most occurrences 
        in the sample as the value of the leaf node
        """
        label_counts = collections.Counter(targets)
        major_label = max(zip(label_counts.values(), label_counts.keys()))
        return major_label[1]

    @staticmethod
    def calc_gini(left_targets, right_targets):
        """
        The classification tree uses the Gini index as an 
        indicator to select the optimal split point
        """
        split_gain = 0
        for targets in [left_targets, right_targets]:
            gini = 1
            # Count how many samples are in each category, 
            # and then calculate gini
            label_counts = collections.Counter(targets)
            for key in label_counts:
                prob = label_counts[key] * 1.0 / len(targets)
                gini -= prob ** 2
            split_gain += len(targets) * 1.0 / (len(left_targets) + len(right_targets)) * gini
        return split_gain

    @staticmethod
    def split_dataset(dataset, targets, split_feature, split_value):
        """
        Divide the sample into left and right parts according to the 
        characteristics and threshold, the left is less than or 
        equal to the threshold, and the right is greater than the threshold
        """
        left_dataset = dataset[dataset[split_feature] <= split_value]
        left_targets = targets[dataset[split_feature] <= split_value]
        right_dataset = dataset[dataset[split_feature] > split_value]
        right_targets = targets[dataset[split_feature] > split_value]
        return left_dataset, right_dataset, left_targets, right_targets

    def predict(self, dataset):
        """Input sample, predict category"""
        res = []
        for _, row in dataset.iterrows():
            pred_list = []
            # Count the prediction results of each tree, 
            # and select the result with the most occurrences 
            # as the final category
            for tree in self.trees:
                pred_list.append(tree.calc_predict_value(row))

            pred_label_counts = collections.Counter(pred_list)
            pred_label = max(zip(pred_label_counts.values(), pred_label_counts.keys()))
            res.append(pred_label[1])
        return np.array(res)

# Run here

In [ ]:
opt_n_estimators = [5,10,25]
opt_max_depth = [10,25,50]
opt_random_state = [2,8,16]
opt_min_samples_split = [10,20,30]

arr_acc_train = []
arr_acc_test = []

for p1,p2,p3,p4 in zip(
    opt_n_estimators, 
    opt_max_depth, 
    opt_random_state, 
    opt_min_samples_split
    ):

  clf = RandomForestClassifier(n_estimators=p1,
                              max_depth=p2,
                              min_samples_split=p4,
                              min_samples_leaf=2,
                              min_split_gain=0.0,
                              colsample_bytree="sqrt",
                              subsample=0.8,
                              random_state=p3)

  x, y = dataFrame.iloc[:, :-1], dataFrame.iloc[:, -1]

  x_train, x_test, y_train, y_test = train_test_split(
      x, y, test_size=0.3, random_state=random.randint(1, 25)
  )

  clf.fit(x_train, y_train)

  from sklearn import metrics
  print(metrics.accuracy_score(y_train, clf.predict(x_train)))
  print(metrics.accuracy_score(y_test, clf.predict(x_test)))
  print(clf.feature_importances_)
  for tree in clf.trees:
    print(tree.describe_tree())
    print(len(tree.describe_tree()))
    sk_tree.plot_tree(tree,filled = True);
    

0.7430939226519337
0.7371794871794872
{'MaintenanceCost': 13, 'Lug_boot': 18, 'BuyPrice': 4, 'Safety': 2}
{split_feature:MaintenanceCost,split_value:2,left_tree:{split_feature:MaintenanceCost,split_value:0,left_tree:{split_feature:Lug_boot,split_value:1,left_tree:{split_feature:Lug_boot,split_value:0,left_tree:{leaf_value:0},right_tree:{leaf_value:0}},right_tree:{leaf_value:0}},right_tree:{split_feature:Lug_boot,split_value:0,left_tree:{split_feature:MaintenanceCost,split_value:1,left_tree:{leaf_value:1},right_tree:{leaf_value:0}},right_tree:{split_feature:MaintenanceCost,split_value:1,left_tree:{split_feature:Lug_boot,split_value:1,left_tree:{leaf_value:1},right_tree:{leaf_value:0}},right_tree:{split_feature:Lug_boot,split_value:1,left_tree:{leaf_value:0},right_tree:{leaf_value:0}}}}},right_tree:{leaf_value:0}}
717


TypeError: ignored